<a href="https://colab.research.google.com/github/YiyunLei/NLP-Final-Project/blob/main/DATA_MSML641_final_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd

In [28]:
# Load datasets from google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
dataset_dir = "/content/drive/MyDrive/NLP_Final_Project/datasets/CPED"

In [30]:
# input - file path
# output - dataframe

def read_df(infile):
  return pd.read_csv(infile)

In [31]:
# to call corresponding dataset, use: sources["xxx"], choose from "test", "train", or "valid"
sources = { 
    "test":  dataset_dir+"/test_split.csv", 
    "train": dataset_dir+"/train_split.csv", 
    "valid": dataset_dir+"/valid_split.csv"
}
read_df(sources["train"])

,TV_ID,Dialogue_ID,Utterance_ID,Speaker,Gender,Age,Neuroticism,Extraversion,Openness,Agreeableness,Conscientiousness,Scene,FacePosition_LU,FacePosition_RD,Sentiment,Emotion,DA,Utterance
0,1,01_000,01_000_000,童文洁,female,middle-aged,high,high,low,low,high,other-venue,108_136,156_202,neutral,neutral,greeting,真巧
1,1,01_000,01_000_001,童文洁,female,middle-aged,high,high,low,low,high,other-venue,193_144,253_197,neutral,neutral,greeting,车没事了
2,1,01_000,01_000_002,刘静,female,middle-aged,low,high,high,high,high,other-venue,0_0,0_0,neutral,neutral,answer,是你呀 没事了没事 谢谢你
3,1,01_000,01_000_003,童文洁,female,middle-aged,high,high,low,low,high,other-venue,0_0,0_0,neutral,neutral,question,没事没事 你也去春风
4,1,01_001,01_001_000,刘静,female,middle-aged,low,high,high,high,high,other-venue,0_0,0_0,neutral,neutral,answer,对
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94182,26,26_265,26_265_019,丁香,female,young,low,low,high,high,high,restaurant,276_57,438_293,neutral,neutral,statement-opinion,不过你要是真有勇气离开北京
94183,26,26_265,26_265_020,丁香,female,young,low,low,high,high,high,restaurant,286_83,447_298,neutral,neutral,statement-opinion,我反倒对你刮目相看
94184,26,26_265,26_265_021,丁香,female,young,low,low,high,high,high,restaurant,268_78,425_299,neutral,neutral,statement-non-opinion,可你走不了
94185,26,26_265,26_265_022,丁香,female,young,low,low,high,high,high,restaurant,0_0,0_0,neutral,neutral,statement-non-opinion,我真得走了


Model

Data: 
```
"Speaker" - String
"Utterance" - String
```

Label:
```
"OCEAN" - -1, 0, 1
```



In [32]:
# Data preprocess
# Input: dataframe
# Function: One-hot encode OCEAN: -1: low, 0: unknown, 1: high
#       delete "其他other"
# Output: y = OCEAN, X = [Speaker, and Utterance]

from sklearn.model_selection import train_test_split

def preprocess(df):
  df = df[df['Speaker'] != '其他'] # delete "其他other"

  big_5 = ['Neuroticism', 'Extraversion', 'Openness', 'Agreeableness', 'Conscientiousness']
  for column_name in big_5:
    df[column_name].replace('high', 1, inplace=True)
    df[column_name].replace('low', -1, inplace=True)
    df[column_name].replace('unknown', 0, inplace=True)
  
  X = df[["Speaker", "Utterance"]]
  y = df[['Neuroticism', 'Extraversion', 'Openness', 'Agreeableness', 'Conscientiousness']]

  return X, y

In [33]:
preprocess(read_df(sources["test"]))

<ipython-input-32-b29252aa4c2c>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name].replace('high', 1, inplace=True)
<ipython-input-32-b29252aa4c2c>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name].replace('low', -1, inplace=True)
<ipython-input-32-b29252aa4c2c>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name].replace('unknown', 0, inplace=True)


(      Speaker          Utterance
 0         林宛瑜              好了没事了
 1         林宛瑜               谢谢你哦
 2         陆展博                没关系
 3         林宛瑜                  喂
 4         林宛瑜                没事吧
 ...       ...                ...
 27430      刘梅           你做不了四菜一汤
 27431      刘梅            你可以做炸酱面
 27432      刘梅               总而言之
 27433      刘梅  洋快餐在我们孩子的食谱里一笔勾销了
 27435      刘梅                  对
 
 [23780 rows x 2 columns],
        Neuroticism  Extraversion  Openness  Agreeableness  Conscientiousness
 0               -1             1         1              1                  1
 1               -1             1         1              1                  1
 2                1            -1         1              1                  1
 3               -1             1         1              1                  1
 4               -1             1         1              1                  1
 ...            ...           ...       ...            ...                ...
 27430           